In [1]:
import pandas as pd
import numpy as np

from random import randint
from time import sleep
import datetime
import random

import os
import sys

sys.path.append(r"C:\Users\Guillermo\Desktop\Python/05. Lib gestión pública")

import redapy

###  TEST 1
Testing that all Redatam's variables are well scrapped and cleaned by redapy at ditrict level

In [2]:
## creating a df with all 2017 census variables
ruta=r'C:\Users\Guillermo\Desktop\Python\05. Lib gestión pública'

var=pd.read_csv(ruta+'\INEI2017\VAR.csv',delimiter=";")
var_u=(var.drop_duplicates("VAR_COMPLETO")[['VAR_COMPLETO','VAR']].copy())
var[var.CATEGORIA.str.contains('inter')]

,VAR_COMPLETO,VAR,NUM_CATEGORIA,CATEGORIA
204,Hogar.C3P213,C3P213,1,Sí tiene conexión a internet
205,Hogar.C3P213,C3P213,2,No tiene conexión a internet


In [3]:
URL_d="https://censos2017.inei.gob.pe/bininei/RpWebStats.exe/CmdSet?BASE=CPV2017DI&ITEM=PROGRED&lang=esp"
URL_m="https://censos2017.inei.gob.pe/bininei/RpWebStats.exe/CmdSet?BASE=CPV2017&ITEM=PROGRED&lang=esp"

chro_url = r'C:\\Users\\Guillermo\\Desktop\\chromedriver.exe'

In [6]:
def test1(var_list, query_url, service_url, area_break=None, selection=None):
    df=pd.DataFrame({'var0':var_list,'error': False,'time_e':'','time_c':''})  
    
    for var_name in var_list:
        delay = randint(1,5)
        sleep(delay)
        try:
            query=redapy.query_final(tipo="Frequency",
                             var1=var_name,
                             area_break=area_break,
                             selection=selection)
            df_e, t1=redapy.make_query_2017(query, query_url, service_url, test=True)
            
            df.loc[df['var0'] == var_name, 'time_e'] = f'{t1.total_seconds():.2f}'
            
            try:
                begin_time_c = datetime.datetime.now()
                redapy.frequency(df_e)
                t2 = datetime.datetime.now() - begin_time_c
                df.loc[df['var0'] == var_name, 'time_c'] = f'{t2.total_seconds():.2f}'
            except:
                df.loc[df['var0'] == var_name, 'time_c'] = 'Error during cleanning'
            
        except Exception as e:
            df.loc[df['var0'] == var_name, 'error'] = True
            df.loc[df['var0'] == var_name, 'time_e'] = ''
            df.loc[df['var0'] == var_name, 'time_c'] = ''
            print(f"Error querying variable '{var_name}': {e}")
    
    return df


In [32]:
## Test1 a nivel de distritos
var_ul=var_u.VAR_COMPLETO.tolist()

df_t1=test1(var_ul, URL_d, chro_url, area_break=["Distrito"],selection=["Provinci 1501"])
df_t1.to_csv(ruta+'\\Notebooks\\Test_results\\test_1.csv', index=False)

Scrapeo iniciado
Se cargó página REDATAM con éxito
Iniciando scrapeo...
La tabla cargó completamente
Tabla scrapeada con éxito en:
0:00:09.454009
Scrapeo iniciado
Se cargó página REDATAM con éxito
Iniciando scrapeo...
La tabla cargó completamente
Tabla scrapeada con éxito en:
0:00:06.514645


In [ ]:
##Test2 a nivel de mazanas
df_t1=test1(var_ul, URL_m, chro_url, area_break=["Manzana"],selection=["Distrito 150108"])
df_t1.to_csv(ruta+'\\Notebooks\\Test_results\\test_1_m.csv', index=False)

In [7]:
##Test3 a nivel de mazanas 
def test1_iter_mz(var_name1, ubigeo_list, query_url, service_url, area_break=None):
    df=pd.DataFrame({'ubi0':ubigeo_list,'error': False,'time_e':'','time_c':''})  
    
    for selec_name in ubigeo_list:
        delay = randint(1,4)
        sleep(delay)
        try:
            query=redapy.query_final(tipo="Frequency",
                                     var1=var_name1,
                                     area_break=area_break,
                                     selection=['Distrito '+str(selec_name)])
            
            df_e, t1=redapy.make_query_2017(query, query_url, service_url, test=True)
            
            df.loc[df['ubi0'] == selec_name, 'time_e'] = f'{t1.total_seconds():.2f}'
            
            try:
                begin_time_c = datetime.datetime.now()
                redapy.frequency(df_e)
                t2 = datetime.datetime.now() - begin_time_c
                df.loc[df['ubi0'] == selec_name, 'time_c'] = f'{t2.total_seconds():.2f}'
            except:
                df.loc[df['ubi0'] == selec_name, 'time_c'] = 'Error during cleanning'
            
        except Exception as e:
            df.loc[df['ubi0'] == selec_name, 'error'] = True
            df.loc[df['ubi0'] == selec_name, 'time_e'] = ''
            df.loc[df['ubi0'] == selec_name, 'time_c'] = ''
            print(f"Error querying variable '{var_name}': {e}")
    return df

ubigeo_list1=["15010"+str(p) for p in list(range(1,10))]
ubigeo_list1=ubigeo_list1+["1501"+str(p) for p in list(range(10,44))]
ubigeo_list1

df_t3=test1_iter_mz(var_name1="Hogar.C3P213", ubigeo_list=ubigeo_list1, query_url=URL_m, service_url=chro_url, area_break=["Manzana"])
df_t3.to_csv(ruta+'\\Notebooks\\Test_results\\test_3_m.csv', index=False)

Scrapeo iniciado
Se cargó página REDATAM con éxito
Iniciando scrapeo...
La tabla cargó completamente
Tabla scrapeada con éxito en:
0:00:21.570997
Scrapeo iniciado
Se cargó página REDATAM con éxito
Iniciando scrapeo...
La tabla cargó completamente
Tabla scrapeada con éxito en:
0:00:13.283471
Scrapeo iniciado
Se cargó página REDATAM con éxito
Iniciando scrapeo...
La tabla cargó completamente
Tabla scrapeada con éxito en:
0:01:00.275689
Scrapeo iniciado
Se cargó página REDATAM con éxito
Iniciando scrapeo...
La tabla cargó completamente
Tabla scrapeada con éxito en:
0:00:07.344940
Scrapeo iniciado
Se cargó página REDATAM con éxito
Iniciando scrapeo...
La tabla cargó completamente
Tabla scrapeada con éxito en:
0:00:07.712043
Scrapeo iniciado
Se cargó página REDATAM con éxito
Iniciando scrapeo...
La tabla cargó completamente
Tabla scrapeada con éxito en:
0:00:42.978211
Scrapeo iniciado
Se cargó página REDATAM con éxito
Iniciando scrapeo...
La tabla cargó completamente
Tabla scrapeada con éxi

####  Crosstab

In [4]:
##test4
def new_var(df,var_list2, num_new_var=5):
    
    df=df.loc[df.index.repeat(num_new_var)].reset_index(drop=True)

    for p in list(range(0,num_new_var)):
        var_list2_s=random.sample(var_list2,num_new_var)        
        df.iloc[p]=["",False,'','',var_list2_s[p]]
    df=df.drop(["var0"], axis=1).set_index("var1")
    
    return df

def test3(var_list,var_list2, query_url, service_url, query_option=None, area_break=None, selection=None):
    
    df=pd.DataFrame({'var0':var_list,'error': False,'time_e':'','time_c':'','var1':''})
    df=df.groupby("var0").apply(lambda _df:new_var(_df,var_list2,num_new_var=3)).reset_index()
      
    
    for var_name,var_name_2 in df.set_index('var0')['var1'].items():
        delay = randint(10,45)
        sleep(delay)
        #print(var_name,var_name_2)
        try:
            query=redapy.query_final(tipo=query_option,
                                         var1=var_name,
                                         var2=var_name_2,
                                         area_break=area_break,
                                         selection=selection)
            df_e, t1=redapy.make_query_2017(query, query_url, service_url, test=True)

            df.loc[(df['var0'] == var_name)&(df['var1'] == var_name_2), 'time_e'] = f'{t1.total_seconds():.2f}'

            try:
                begin_time_c = datetime.datetime.now()
                redapy.cross_table(df_e)
                t2 = datetime.datetime.now() - begin_time_c
                df.loc[(df['var0'] == var_name)&(df['var1'] == var_name_2), 'time_c'] = f'{t2.total_seconds():.2f}'
            except:
                df.loc[(df['var0'] == var_name)&(df['var1'] == var_name_2), 'time_c'] = 'Error during cleanning'

        except Exception as e:
            df.loc[(df['var0'] == var_name)&(df['var1'] == var_name_2), 'error'] = True
            df.loc[(df['var0'] == var_name)&(df['var1'] == var_name_2), 'time_e'] = ''
            df.loc[(df['var0'] == var_name)&(df['var1'] == var_name_2), 'time_c'] = ''
            print(f"Error querying variable '{var_name}': {e}")
    
    return df

# Hogar
# Poblacio
var_u["cedula"]=var_u.VAR_COMPLETO.str.split(".",expand=True)[0]
var_ul1=var_u.query('cedula=="Poblacio"').VAR_COMPLETO.tolist()
var_ul2=var_u.VAR_COMPLETO.tolist()

df_t4=test3(var_ul1,var_ul2, URL_d, chro_url, query_option="Crosstab", area_break=["Distrito"],selection=["Provinci 1501"])
df_t4.to_csv(ruta+'\\Notebooks\\Test_results\\test_3_d_LM3.csv', index=False)

Scrapeo iniciado
Se cargó página REDATAM con éxito
La tabla cargó completamente
Tabla scrapeada con éxito en: 0:00:10.048891
Scrapeo iniciado
Se cargó página REDATAM con éxito
La tabla cargó completamente
Tabla scrapeada con éxito en: 0:00:09.650276
Scrapeo iniciado
Se cargó página REDATAM con éxito
La tabla cargó completamente
Tabla scrapeada con éxito en: 0:00:08.237130
Scrapeo iniciado
Se cargó página REDATAM con éxito
La tabla cargó completamente
Tabla scrapeada con éxito en: 0:00:08.287030
Scrapeo iniciado
Se cargó página REDATAM con éxito
La tabla cargó completamente
Tabla scrapeada con éxito en: 0:00:06.352323
Scrapeo iniciado
Se cargó página REDATAM con éxito
La tabla cargó completamente
Tabla scrapeada con éxito en: 0:00:06.573541
Scrapeo iniciado
Se cargó página REDATAM con éxito
La tabla cargó completamente
Tabla scrapeada con éxito en: 0:00:10.199076
Scrapeo iniciado
Se cargó página REDATAM con éxito
La tabla cargó completamente
Tabla scrapeada con éxito en: 0:00:07.623960


In [5]:
dft4_1=pd.read_csv(ruta+'\\Notebooks\\Test_results\\test_3_d_LM.csv')
dft4_2=pd.read_csv(ruta+'\\Notebooks\\Test_results\\test_3_d_LM2.csv')
dft4_3=pd.read_csv(ruta+'\\Notebooks\\Test_results\\test_3_d_LM3.csv')
df_t4=pd.concat([dft4_1,dft4_2,dft4_3],axis=0)
df_t4.to_csv(ruta+'\\Notebooks\\Test_results\\test_3_d_LM0.csv', index=False)

In [ ]:
a=df_t4.query('error==False').copy()
a['time_e']=a['time_e'].astype(float)
a['time_c']=a['time_c'].astype(float)
#a.sort_values('time_e')
a